# 1. Import libraries

In [2]:
import pandas as pd

In [3]:
superstore_df = pd.read_csv('superstore_raw.csv')

# 2. Check the data

In [4]:
print(superstore_df.head())

   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
1       2  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
2       3  CA-2017-138688  12/06/2017  16/06/2017    Second Class    DV-13045   
3       4  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   
4       5  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   

     Customer Name    Segment        Country             City       State  \
0      Claire Gute   Consumer  United States        Henderson    Kentucky   
1      Claire Gute   Consumer  United States        Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United States      Los Angeles  California   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   

   Postal Code Region       Product ID         Cat

# 3. Define tables and primary/foreign keys (PK/FK)

customers
* customer_id (PK)
* customer_name
* segment

products
* product_id (PK)
* product_name
* category
* sub_category

regions
* region_id (PK) - will be created
* country
* state
* city
* postal_code
* region

orders
* order_id (PK)
* order_date
* ship_date
* ship_mode
* customer_id (FK)
* region_id (FK)

order_items
* row_id (PK)
* order_id (FK)
* product_id (FK)
* sales

# 4. Define relational model

4.1 customers → orders (1:N relationship)
A single customer can place multiple orders, but each order belongs to exactly one customer.

* customers.customer_id is the primary key
* orders.customer_id is a foreign key referencing it

Why this relationship exists:  
In the raw CSV, the same customer appears many times across different rows. Storing customers in a separate table prevents duplication of names and segments, and allows customer‑level analysis.

4.2 regions → orders (1:N relationship)
Each order is associated with one geographic location (city, state, region), but a region can contain many orders.
* regions.region_id is the primary key
* orders.region_id is a foreign key

Why this relationship exists:  
Location fields repeat heavily in the dataset. Creating a dedicated regions table avoids redundancy and enables geographic reporting (sales by state, region, etc.).

4.3 orders → order_items (1:N relationship)
An order can contain multiple products, but each order item belongs to exactly one order.
* orders.order_id is the primary key
* order_items.order_id is a foreign key

Why this relationship exists:  
In the CSV, each Order ID appears multiple times — once per product purchased. This relationship reflects that structure and allows detailed line‑item analysis.